# 
<a id="data-prep"></a>
<div style="background-color: #000D5B; color: white; text-align: center; padding: 6px 0 22px 0">
    <h3 style="background-color: #000D5B; color: white; text-align: left">RMIT School of Computer Science and Technology</h3>
    <br/>
    <h1>COSC3007: Deep Learning</h1>
    <h2>Assignment 2: Stance Twitter Sentiment Analysis and Detection </h2>
    <h2> [2] MODELLING AND MODEL EVALUATIONS </h2>

</div>

# Import Libs

In [1]:
import pandas as pd 
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

import numpy as np

# 1 Load Data

In [2]:
train_df = pd.read_csv("./StanceDataset/processed_train.csv", encoding = "ISO-8859-1", engine="python").drop(columns=['Unnamed: 0'])
test_df = pd.read_csv("./StanceDataset/processed_test.csv", encoding = "ISO-8859-1", engine="python").drop(columns=['Unnamed: 0'])

# 2 Prepare label and fit data

## Split test and val

In [3]:
test_df, val_df = train_test_split(test_df, test_size=0.5, random_state=42)

## Tweets

In [4]:
MAX_LENGTH = 200

# Train
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['processed_tweet'])
train_sequences = tokenizer.texts_to_sequences(train_df['processed_tweet'])

# Test
tokenizer2 = Tokenizer()
tokenizer2.fit_on_texts(train_df['processed_tweet'])
test_sequences = tokenizer2.texts_to_sequences(test_df['processed_tweet'])

# Val
tokenizer3 = Tokenizer()
tokenizer3.fit_on_texts(train_df['processed_tweet'])
val_sequences = tokenizer3.texts_to_sequences(val_df['processed_tweet'])

train_pad = pad_sequences(train_sequences, maxlen=MAX_LENGTH, padding='post')
test_pad = pad_sequences(test_sequences, maxlen=MAX_LENGTH, padding='post')
val_pad = pad_sequences(val_sequences, maxlen=MAX_LENGTH, padding='post')

## Stances

In [5]:
def categorized_label(df, label_name):
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(df[label_name])
    categorical_labels = to_categorical(encoded_labels)
    return categorical_labels

In [6]:
train_labels = categorized_label(train_df, "Stance")
test_labels = categorized_label(test_df, "Stance")
val_labels = categorized_label(val_df, "Stance")

# 3 Create Model

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

In [8]:
# Build the LSTM model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=16, input_length=200))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))  # 3 classes: FAVOUR, AGAINST, NEITHER

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

2023-12-19 13:12:34.647460: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2023-12-19 13:12:34.647479: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2023-12-19 13:12:34.647485: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2023-12-19 13:12:34.647515: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-19 13:12:34.647530: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [9]:
history = model.fit(train_pad, train_labels, epochs=10, validation_data=(val_pad, val_labels), batch_size=32)

Epoch 1/10


2023-12-19 13:12:35.630744: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


10/92 [==>...........................] - ETA: 2:06:51 - loss: 1.0825 - accuracy: 0.4594